# Portfolio Optimization Using Python

We will use the wrds api to get data on mutual funds. Then we will use mean-variance and minimum-variance optimization to construct a portfolio of funds

### Setup

In [1]:
# importing all relevant packages and modules
import numpy as np
import pandas as pd
import wrds

### Connecting to WRDS and getting the data

In [2]:
while True:
    try:
        conn = wrds.Connection()
    except:
        continue
    else:
        break

Enter your WRDS username [ujjwa]:ujjwalm
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: n
You can create this file yourself at any time
with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
start_date = '2012-12-17'
end_date = '2022-12-16'

In [4]:
fund_list = ['VOO', 'VNQ']
fund_list_str = '(' + ','.join(['\''+ticker+'\'' for ticker in fund_list]) + ')'
fund_list_str

"('VOO','VNQ')"

In [5]:
# Get fund numbers for all the funds
query_fundno = \
'''
SELECT
    nasdaq, crsp_fundno
FROM
    crsp.mfdbname
WHERE
     nasdaq in ''' + fund_list_str +\
'''
GROUP BY
    nasdaq, crsp_fundno
'''
fundno_df = conn.raw_sql(query_fundno)
fundno_df['nasdaq'] = pd.Categorical(fundno_df.nasdaq,
                                     categories = fund_list,
                                     ordered=True
                                    )
fundno_df = fundno_df.sort_values('nasdaq')
fundno_df

,nasdaq,crsp_fundno
1,VOO,50485.0
0,VNQ,31446.0


In [6]:
fundno_str = '('+ ','.join(list(fundno_df.crsp_fundno.values.astype(str)))+')'

In [7]:
query_mret = \
'''
SELECT 
    *
FROM
    crsp.monthly_returns
WHERE
    crsp_fundno in ''' + fundno_str + 'and' + \
'''
    caldt >= ''' + '\'' + start_date + '\'' + 'and\n    caldt <=' + '\'' + end_date + '\'' 

In [8]:
mret_df = conn.raw_sql(query_mret)
mret_df = mret_df.pivot(index = 'caldt', columns = 'crsp_fundno').droplevel(0,axis = 1)
column_dict = {fundno_df.crsp_fundno[i]:fundno_df.nasdaq[i] for i in range(len(fundno_df))}
mret_df.rename(columns=column_dict, inplace=True)
mret_df.reset_index(inplace=True)
mret_df = pd.concat([mret_df['caldt'],mret_df[fundno_df.nasdaq]], axis=1)
mret_df

,caldt,VOO,VNQ
0,2012-12-31,0.009222,0.037211
1,2013-01-31,0.051848,0.037240
2,2013-02-28,0.013417,0.012456
3,2013-03-28,0.037383,0.028973
4,2013-04-30,0.019233,0.067045
...,...,...,...
113,2022-05-31,0.001795,-0.046124
114,2022-06-30,-0.082567,-0.075120
115,2022-07-29,0.092183,0.086656
116,2022-08-31,-0.040802,-0.060239


### Setting up data for the portfolio optimization

In [9]:
# Risk free rate
rf_rate = 0.03/12 # div by 12, given we are using monthly returns for the calculations

In [10]:
if mret_df.isna().sum().sum()!=0:
    print(mret_df.isna().sum())
    mret_df.dropna(inplace=True)

In [11]:
if 'caldt' in mret_df.columns:
    mret_df.drop(['caldt'],axis=1, inplace=True)

# a numpy array with shape (1, number of stocks)
fund_exp_ret = np.array([mret_df.mean(axis=0).to_numpy()]) 

# You can also provide your own expected returns
#fund_exp_ret = np.array([[0.02, 0.01, 0.01, 0.007]])

# a numpy array with shape (number of stocks, number of stocks)
fund_cov = mret_df.cov().to_numpy()

In [12]:
['{:.2%}'.format(i) for i in fund_exp_ret[0]]

['1.04%', '0.64%']

In [13]:
fund_cov

array([[0.00175109, 0.00144084],
       [0.00144084, 0.00238176]])

In [14]:
mret_df.corr()

,VOO,VNQ
VOO,1.000000,0.705526
VNQ,0.705526,1.000000


### Mean variance optimizer

In [19]:
def mean_variance_optimizer(returns, rf_rate, cov):
    
    """
    returns should be a numpy array with shape (1, number of stocks)
    rf_rate should be a float
    cov should be a numpy array with shape (number of stocks, number of stocks)
    """
    
    # Setup
    from scipy.optimize import minimize, Bounds
    
    # Creating a starting portfolio weight with n_col = 1 and n_row = number of stocks
    wt = np.ones((returns.shape[1],1))*(1.0/returns.shape[1])
    
    # Defining the objective function for the minimization optimization 
    def obj_func_mean_variance(wt, returns, rf_rate, cov):
        wt_ret = returns@wt - rf_rate
        std_dev = (wt.T@cov@wt)**0.5
        sharpe_ratio = wt_ret/std_dev
        return -sharpe_ratio
    
    # Setting variable bounds between 0 and 1 i.e. the optimizer doesnt account for shorting or leveraging
    var_bounds = Bounds(0,1)
    
    # Setting sum of portfolio weights to 1
    var_constraints = {'type':'eq',
                       'fun': lambda wt: 1.0 - np.sum(wt)
                      }
    
    # calling the minizime function from the scipy package
    mean_variance = minimize(obj_func_mean_variance,
                             wt,
                             args=(returns, rf_rate, cov),
                             bounds=var_bounds,
                             constraints=var_constraints
                            )
    
    optimal_sharpe_ratio = -mean_variance.fun
    optimal_port_wt = np.array([mean_variance.x])
    optimal_expected_return = optimal_port_wt@returns.T
    optimal_std_dev = (optimal_port_wt@cov@optimal_port_wt.T)**0.5
    
    print('Mean variance optimization results\n')
    print('The optimal weights of the portfolio are:')
    print(['{:.2%}'.format(i) for i in optimal_port_wt[0]])
    print('Optimal expected return is {:.2%}'.format(optimal_expected_return[0,0]))
    print('Optimal std dev is {:.2f}'.format(optimal_std_dev[0,0]))
    print('Optimal sharpe ratio is {:.2f}'.format(optimal_sharpe_ratio[0]))
    
    return optimal_port_wt, optimal_expected_return, optimal_std_dev, optimal_sharpe_ratio

### Minimum variance optimization

In [20]:
def min_variance_optimizer(returns, rf_rate, cov):
    
    """
    returns should be a numpy array with shape (1, number of stocks)
    rf_rate should be a float
    cov should be a numpy array with shape (number of stocks, number of stocks)
    """
    
    # Setup
    from scipy.optimize import minimize, Bounds
    
    # Creating a starting portfolio weight with n_col = 1 and n_row = number of stocks
    wt = np.ones((returns.shape[1],1))*(1.0/returns.shape[1])
    
    # Defining the objective function for the minimization optimization 
    def obj_func_min_var(wt,cov):
        return (wt.T@cov@wt)**0.5
    
    # Setting variable bounds between 0 and 1 i.e. the optimizer doesnt account for shorting or leveraging
    var_bounds = Bounds(0,1)
    
    # Setting sum of portfolio weights to 1
    var_constraints = {'type':'eq',
                       'fun': lambda wt: 1.0 - np.sum(wt)
                      }
    
    # calling the minizime function from the scipy package
    min_var = minimize(obj_func_min_var,
                             wt,
                             args=(cov),
                             bounds=var_bounds,
                             constraints=var_constraints
                            )
    
    min_var_port_wt = np.array([min_var.x])
    min_var_expected_return = min_var_port_wt@returns.T
    min_var_std_dev = min_var.fun
    min_var_sharpe_ratio = (min_var_expected_return-rf_rate)/min_var_std_dev
    
    print('Minimum variance optimization results\n')
    print('The optimal weights of the portfolio are:')
    print(['{:.2%}'.format(i) for i in min_var_port_wt[0]])
    print('Optimal expected return is {:.2%}'.format(min_var_expected_return[0,0]))
    print('Optimal std dev is {:.2f}'.format(min_var_std_dev))
    print('Optimal sharpe ratio is {:.2f}'.format(min_var_sharpe_ratio[0,0]))
    
    return min_var_port_wt,min_var_expected_return,min_var_std_dev,min_var_sharpe_ratio

### Running the optimization algorithm

In [21]:
mean_var_wt, mean_var_ret, mean_var_sd, mean_var_sr = mean_variance_optimizer(fund_exp_ret,rf_rate,fund_cov)

Mean variance optimization results

The optimal weights of the portfolio are:
['100.00%', '0.00%']
Optimal expected return is 1.04%
Optimal std dev is 0.04
Optimal sharpe ratio is 0.19


In [22]:
min_var_wt, min_var_ret, min_var_sd, min_var_sr = min_variance_optimizer(fund_exp_ret,rf_rate,fund_cov)

Minimum variance optimization results

The optimal weights of the portfolio are:
['75.66%', '24.34%']
Optimal expected return is 0.94%
Optimal std dev is 0.04
Optimal sharpe ratio is 0.17
